In [1]:
import numpy as np#for arrays and matrices
import pandas as pd#for web apis and other functionalities
import matplotlib.pyplot as plt#graph polt and classifications to plot graphs
import torch#frame work used to compute graphs 
from torchvision import datasets, transforms, models  # datsets  , transforms
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn as nn #to work with nn outputs
import torch.nn.functional as F #to work with nn functionalities
from datetime import datetime
import zipfile
import os
from torchsummary import summary
from PIL import Image
import torchvision.transforms.functional as TF


loading the data

In [2]:

!wget --no-check-certificate \
    "https://data.mendeley.com/public-files/datasets/tywbtsjrjv/files/b4e3a32f-c0bd-4060-81e9-6144231f2520/file_downloaded" \
    -O "../data/Dataset.zip"
    

--2022-12-26 10:59:28--  https://data.mendeley.com/public-files/datasets/tywbtsjrjv/files/b4e3a32f-c0bd-4060-81e9-6144231f2520/file_downloaded
Resolving data.mendeley.com (data.mendeley.com)... 162.159.133.86, 162.159.130.86
Connecting to data.mendeley.com (data.mendeley.com)|162.159.133.86|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://prod-dcd-datasets-public-files-eu-west-1.s3.eu-west-1.amazonaws.com/349ac012-2948-4172-bbba-3bf8f76596fd [following]
--2022-12-26 10:59:29--  https://prod-dcd-datasets-public-files-eu-west-1.s3.eu-west-1.amazonaws.com/349ac012-2948-4172-bbba-3bf8f76596fd
Resolving prod-dcd-datasets-public-files-eu-west-1.s3.eu-west-1.amazonaws.com (prod-dcd-datasets-public-files-eu-west-1.s3.eu-west-1.amazonaws.com)... 52.218.30.72, 52.218.109.216, 52.92.2.2, ...
Connecting to prod-dcd-datasets-public-files-eu-west-1.s3.eu-west-1.amazonaws.com (prod-dcd-datasets-public-files-eu-west-1.s3.eu-west-1.amazonaws.com)|52.218.30.72|:443.

In [3]:
zip_ref = zipfile.ZipFile('../data/Dataset.zip', 'r') #Opens the zip file in read mode
zip_ref.extractall('../data') #Extracts the files into the /tmp folder